In [118]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import timedelta
from datetime import datetime
import pytz
from datetime import timezone
import time
import sqlalchemy
from sqlalchemy import create_engine
import psycopg2 
import io
from urllib.parse import quote  

In [84]:
data = '/content/drive/MyDrive/MockUp.csv'
df = pd.read_csv(data)

In [94]:
df.head()

,Service Request (SR) Number,SR Description,Method Received,SR Status,Status Change Date,Created Date,Last Update Date,Close Date,SR Location,Street Number,Street Name,City,Zip Code,County,State Plane X Coordinate,State Plane Y Coordinate,Latitude Coordinate,Longitude Coordinate,(Latitude.Longitude),Council District,Map Page,Map Tile,Created Date DT,Request Year,Department,Closed Date DT,Open Time,Open Time delta,Total Seconds
0,16-00033203,Animal Control - Assistance Request,Phone,Closed,10/24/2019 12:08:44 PM,02/14/2016 12:01:22 PM,10/24/2019 12:08:43 PM,10/24/2019 12:08:44 PM,"4713 CLAWSON RD, AUSTIN, TX 78745",4713.0,CLAWSON,AUSTIN,78745.0,TRAVIS,3.100924e+06,1.005352e+07,30.221533,-97.786457,"(30.22153337, -97.78645728)",5.0,644A,MG18,2016-02-14 12:01:22,2016,Animal Services,2019-10-24 12:08:44,1348 days 00:07:22,1348 days 00:07:22,116467642.0
1,16-00067102,Animal Control - Assistance Request,Phone,Closed,10/24/2019 09:47:25 AM,03/24/2016 03:15:25 PM,10/24/2019 09:47:24 AM,10/24/2019 09:47:25 AM,"7515 BETHUNE AVE, AUSTIN, TX 78752",7515.0,BETHUNE,AUSTIN,78752.0,TRAVIS,3.128699e+06,1.009526e+07,30.334498,-97.695409,"(30.33449829, -97.69540895)",4.0,526X,ML28,2016-03-24 15:15:25,2016,Animal Services,2019-10-24 09:47:25,1308 days 18:32:00,1308 days 18:32:00,113077920.0
2,16-00112695,Animal Control - Assistance Request,Phone,Closed,10/24/2019 12:11:34 PM,05/11/2016 02:34:25 PM,10/24/2019 12:11:34 PM,10/24/2019 12:11:34 PM,"4713 CLAWSON RD, AUSTIN, TX 78745",4713.0,CLAWSON,AUSTIN,78745.0,TRAVIS,3.100924e+06,1.005352e+07,30.221533,-97.786457,"(30.22153337, -97.78645728)",5.0,644A,MG18,2016-05-11 14:34:25,2016,Animal Services,2019-10-24 12:11:34,1260 days 21:37:09,1260 days 21:37:09,108941829.0
3,18-00427753,Austin Code - Request Code Officer,Phone,Closed,12/15/2018 02:55:45 PM,12/15/2018 02:45:01 PM,12/02/2019 12:40:44 PM,12/15/2018 02:55:45 PM,"3408 CLARKSBURG DR, AUSTIN, TX 78745",3408.0,CLARKSBURG,Austin,78745.0,TRAVIS,3.086400e+06,1.004795e+07,30.207125,-97.832827,"(30.20712463, -97.83282719)",5.0,643N,ME17,2018-12-15 14:45:01,2018,Austin Code,2018-12-15 14:55:45,0 days 00:10:44,0 days 00:10:44,644.0
4,18-00440567,Pavement Failure,Phone,Closed,01/25/2019 08:15:00 PM,12/27/2018 03:08:13 PM,05/26/2021 09:51:52 PM,01/25/2019 08:15:00 PM,"4300 CLAWSON RD, AUSTIN, TX 78704",4300.0,CLAWSON,AUSTIN,78704.0,TRAVIS,3.101555e+06,1.005624e+07,30.228994,-97.784261,"(30.22899356, -97.78426119)",5.0,614W,MG19,2018-12-27 15:08:13,2018,Public Works,2019-01-25 20:15:00,29 days 05:06:47,29 days 05:06:47,2524007.0


In [86]:
df = df.drop(columns='Unnamed: 0')

In [88]:
df["Created Date DT"] = pd.to_datetime(df["Created Date DT"])
df['Closed Date DT'] = pd.to_datetime(df['Closed Date DT'])

In [89]:
df.dtypes

Service Request (SR) Number            object
SR Description                         object
Method Received                        object
SR Status                              object
Status Change Date                     object
Created Date                           object
Last Update Date                       object
Close Date                             object
SR Location                            object
Street Number                         float64
Street Name                            object
City                                   object
Zip Code                              float64
County                                 object
State Plane X Coordinate              float64
State Plane Y Coordinate              float64
Latitude Coordinate                   float64
Longitude Coordinate                  float64
(Latitude.Longitude)                   object
Council District                      float64
Map Page                               object
Map Tile                          

In [90]:
#dt = datetime(2015, 10, 19)
#timestamp = (dt - datetime(1970, 1, 1)).total_seconds()
#df['Open Date UTC'] = df['Created Date DT'].subtract(datetime(1970, 1, 1, 0, 0, 0))
#df['Total Seconds Open'] = df['Open Time'].total_seconds()
df['Open Time delta'] = pd.to_timedelta(df['Open Time'])
df['Total Seconds'] = (df['Open Time delta']).dt.total_seconds()

In [92]:
average_open = df['Total Seconds'].mean()

In [93]:
average_open

3691927.6939125513

In [95]:
print(df.columns)

Index(['Service Request (SR) Number', 'SR Description', 'Method Received',
       'SR Status', 'Status Change Date', 'Created Date', 'Last Update Date',
       'Close Date', 'SR Location', 'Street Number', 'Street Name', 'City',
       'Zip Code', 'County', 'State Plane X Coordinate',
       'State Plane Y Coordinate', 'Latitude Coordinate',
       'Longitude Coordinate', '(Latitude.Longitude)', 'Council District',
       'Map Page', 'Map Tile', 'Created Date DT', 'Request Year', 'Department',
       'Closed Date DT', 'Open Time', 'Open Time delta', 'Total Seconds'],
      dtype='object')


In [96]:
df = df[['Service Request (SR) Number', 'SR Description', 'Department', 'Method Received', 'SR Status', 'Zip Code', 'Council District', 'City', 'County', 'Request Year', 'Open Time', 'Open Time delta', 'Total Seconds', 'Created Date', 'Created Date DT', 'Close Date', 'Closed Date DT', 'Status Change Date', 'Last Update Date', 'Map Page', 'Map Tile', 'State Plane X Coordinate', 'State Plane Y Coordinate', 'Latitude Coordinate', 'Longitude Coordinate', '(Latitude.Longitude)', 'SR Location', 'Street Number', 'Street Name']]

In [97]:
df.head(10)

,Service Request (SR) Number,SR Description,Department,Method Received,SR Status,Zip Code,Council District,City,County,Request Year,Open Time,Open Time delta,Total Seconds,Created Date,Created Date DT,Close Date,Closed Date DT,Status Change Date,Last Update Date,Map Page,Map Tile,State Plane X Coordinate,State Plane Y Coordinate,Latitude Coordinate,Longitude Coordinate,(Latitude.Longitude),SR Location,Street Number,Street Name
0,16-00033203,Animal Control - Assistance Request,Animal Services,Phone,Closed,78745.0,5.0,AUSTIN,TRAVIS,2016,1348 days 00:07:22,1348 days 00:07:22,116467642.0,02/14/2016 12:01:22 PM,2016-02-14 12:01:22,10/24/2019 12:08:44 PM,2019-10-24 12:08:44,10/24/2019 12:08:44 PM,10/24/2019 12:08:43 PM,644A,MG18,3.100924e+06,1.005352e+07,30.221533,-97.786457,"(30.22153337, -97.78645728)","4713 CLAWSON RD, AUSTIN, TX 78745",4713.0,CLAWSON
1,16-00067102,Animal Control - Assistance Request,Animal Services,Phone,Closed,78752.0,4.0,AUSTIN,TRAVIS,2016,1308 days 18:32:00,1308 days 18:32:00,113077920.0,03/24/2016 03:15:25 PM,2016-03-24 15:15:25,10/24/2019 09:47:25 AM,2019-10-24 09:47:25,10/24/2019 09:47:25 AM,10/24/2019 09:47:24 AM,526X,ML28,3.128699e+06,1.009526e+07,30.334498,-97.695409,"(30.33449829, -97.69540895)","7515 BETHUNE AVE, AUSTIN, TX 78752",7515.0,BETHUNE
2,16-00112695,Animal Control - Assistance Request,Animal Services,Phone,Closed,78745.0,5.0,AUSTIN,TRAVIS,2016,1260 days 21:37:09,1260 days 21:37:09,108941829.0,05/11/2016 02:34:25 PM,2016-05-11 14:34:25,10/24/2019 12:11:34 PM,2019-10-24 12:11:34,10/24/2019 12:11:34 PM,10/24/2019 12:11:34 PM,644A,MG18,3.100924e+06,1.005352e+07,30.221533,-97.786457,"(30.22153337, -97.78645728)","4713 CLAWSON RD, AUSTIN, TX 78745",4713.0,CLAWSON
3,18-00427753,Austin Code - Request Code Officer,Austin Code,Phone,Closed,78745.0,5.0,Austin,TRAVIS,2018,0 days 00:10:44,0 days 00:10:44,644.0,12/15/2018 02:45:01 PM,2018-12-15 14:45:01,12/15/2018 02:55:45 PM,2018-12-15 14:55:45,12/15/2018 02:55:45 PM,12/02/2019 12:40:44 PM,643N,ME17,3.086400e+06,1.004795e+07,30.207125,-97.832827,"(30.20712463, -97.83282719)","3408 CLARKSBURG DR, AUSTIN, TX 78745",3408.0,CLARKSBURG
4,18-00440567,Pavement Failure,Public Works,Phone,Closed,78704.0,5.0,AUSTIN,TRAVIS,2018,29 days 05:06:47,29 days 05:06:47,2524007.0,12/27/2018 03:08:13 PM,2018-12-27 15:08:13,01/25/2019 08:15:00 PM,2019-01-25 20:15:00,01/25/2019 08:15:00 PM,05/26/2021 09:51:52 PM,614W,MG19,3.101555e+06,1.005624e+07,30.228994,-97.784261,"(30.22899356, -97.78426119)","4300 CLAWSON RD, AUSTIN, TX 78704",4300.0,CLAWSON
5,14-00074537,Animal Control - Assistance Request,Animal Services,Phone,Closed,78704.0,3.0,AUSTIN,TRAVIS,2014,1993 days 04:08:19,1993 days 04:08:19,172210099.0,04/22/2014 04:31:24 PM,2014-04-22 16:31:24,10/06/2019 08:39:43 PM,2019-10-06 20:39:43,10/06/2019 08:39:43 PM,10/06/2019 08:39:44 PM,614U,MH19,3.107482e+06,1.005789e+07,30.233136,-97.765378,"(30.23313629, -97.76537843)","3200 S 1ST ST, AUSTIN, TX",3200.0,1ST
6,18-00335556,Curb/Gutter Repair,Public Works,Web,Closed,78757.0,7.0,AUSTIN,TRAVIS,2018,270 days 08:53:38,270 days 08:53:38,23360018.0,10/04/2018 02:43:32 PM,2018-10-04 14:43:32,07/01/2019 11:37:10 PM,2019-07-01 23:37:10,07/01/2019 11:37:10 PM,12/29/2019 12:03:01 AM,525X,MJ28,3.116439e+06,1.009587e+07,30.336979,-97.734216,"(30.33697908, -97.7342161)","1800 PEQUENO ST, AUSTIN, TX 78757",1800.0,PEQUENO
7,18-00184170,Animal - Proper Care,Animal Services,Phone,Closed,78741.0,3.0,AUSTIN,TRAVIS,2018,564 days 02:22:18,564 days 02:22:18,48738138.0,06/13/2018 02:35:08 PM,2018-06-13 14:35:08,12/29/2019 04:57:26 PM,2019-12-29 16:57:26,12/29/2019 04:57:26 PM,12/29/2019 04:57:26 PM,645C,MK18,3.119593e+06,1.005529e+07,30.225215,-97.727232,"(30.22521477, -97.72723243)","2504 HUNTWICK DR 504, AUSTIN, TX 78741",2504.0,HUNTWICK
8,17-00026230,Loose Dog,Animal Services,Phone,Closed,78745.0,5.0,AUSTIN,TRAVIS,2017,978 days 08:42:37,978 days 08:42:37,84530557.0,01/31/2017 12:48:08 PM,2017-01-31 12:48:08,10/06/2019 09:30:45 PM,2019-10-06 21:30:45,

In [133]:
#exporting to csv
df.to_csv(path_or_buf='/content/drive/MyDrive/Training.csv')

In [130]:
db_password = 'xx'

In [131]:
engine = create_engine('postgresql://postgres:%s@localhost:5432/final_project)' % quote('xx'))

In [132]:
df.to_sql(name='calls', con=engine)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2615: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  method=method,


OperationalError: ignored